# Predicting biomarkers for depletion of glutathione upon use of paracetamol in RECON 2
Thierry D.G.A. Mondeel & Hans V. Westerhoff <br />
University of Amsterdam 2017

--- 

This is for advanced students or student projects. 

<span style="color:red">**Assignment:**</span> Read the (Geenen, 2012) publication. 

<span style="color:red">**Assignment:**</span> The teaching assistants can supply you with the kinetic model that works in Copasi. Try reproducing some of the results shown in the publication. Especially, the relationship between paracetamol increase and ophthalmic acid and oxoproline. 

<span style="color:red">**Assignment:**</span> The main goal here is to see what we can say about metabolic changes upon paracetamol influx (or modeled as a drain on glutathione for simplicity). Specifically, given the Shlomi et al method. 

Set up Python, load RECON 2, add opthalmic acid reactions (check if needed for RECON 2.2!), add demand reaction for glutathione.

In [ ]:
import cobra
from cobra.solvers import get_solver_name
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import parsimonious
from utils import show_map, findBiomarkers

# Load Recon2
M = cobra.io.load_json_model("models/recon_2_2_simple_medium.json")


# Adding the opthalmic acid pathway to RECON 2
opa_e = Metabolite('opa_e')
opa_e.name = 'Ophthalmic acid'
opa_c = Metabolite('opa_c')
opa_c.name = 'Ophthalmic acid'

# exchange reaction
EX_opa_e = Reaction('EX_opa_e')
EX_opa_e.add_metabolites({opa_e: -1.0})
EX_opa_e.lower_bound = 0; EX_opa_e.upper_bound = 1000

# transport reaction going outward
OPAtrs = Reaction('OPAtrs')
OPAtrs.add_metabolites({opa_e: 1.0,opa_c: -1.0})
OPAtrs.lower_bound = -1000; OPAtrs.upper_bound = 1000

# glutathione synthase: OPA <-> gluAB (CE1661)
gluABs = Reaction('gluABs')
gluABs.add_metabolites({opa_c: -1.0,M.metabolites.gly_c:-1.0,M.metabolites.CE1661_c: 1.0})
gluABs.lower_bound = -1000; gluABs.upper_bound = 1000

M.add_reactions([EX_opa_e,OPAtrs,gluABs])

# Find all exchange and transport reactions
exchanges = [rxn for rxn in M.reactions if rxn.products == [] ]
exchangesIds = [rxn.id for rxn in exchanges]
transport = [rxn for rxn in M.reactions if len(rxn.get_compartments()) > 1]

# make all exchanges in/out have a bound of 0/1000
for rxn in exchanges:
    if rxn.lower_bound < 0:
        rxn.lower_bound = 0
    
    rxn.upper_bound = 1000     
        
# Add reactions to the model
# demand reaction for gthrd[c] 
DM_gthrd_c = Reaction('DM_gthrd_c')
DM_gthrd_c.add_metabolites({M.metabolites.gthrd_c: -1.0})
# ATP reaction
atp_synthetase_c = Reaction('atp_synthetase_c')
atp_synthetase_c.add_metabolites({M.metabolites.atp_c: -1.0, M.metabolites.adp_c:1.0, 
                                  M.metabolites.pi_c:1.0})
atp_synthetase_c.lower_bound = -1000; atp_synthetase_c.upper_bound = 1000;

M.add_reactions([atp_synthetase_c,DM_gthrd_c])

# set objective
M.reactions.DM_gthrd_c.upper_bound = 1 
M.objective = M.reactions.DM_gthrd_c

In [ ]:
model = M.copy()
model.objective = model.reactions.biomass_reaction 

df = findBiomarkers(model,['EX_5oxpro_e','EX_opa_e','EX_glu_L_e',
                          'EX_cys_L_e','EX_gly_e'],mode='drug',mods=['DM_gthrd_c'],cutoff=0); df